# 나만의 자비스! 음성 비서 만들기
OpenAI의 ChatGPT의 **Function Calling** 기능을 활용하면, 아이언맨의 자비스와 같은 인공지능 비서를 만들 수 있습니다.  

![](assets/stt_chatgpt_tts.png)

한국어 대화도 가능합니다. 실시간 대화 기능을 구현하기 위해서 다음의 기술을 활용할 수 있습니다.  

1. 사용자의 음성은 Microsoft Azure가 제공하는 STT(Speech to Text) 서비스를 이용하여 텍스트로 변환합니다.
2. 텍스트는 ChatGPT가 제공하는 Function Calling 기능을 활용하여 이미 작성되어진 API 또는 코드를 선택적으로 수행합니다.
3. 수행 결과는 다시 Microsoft Azure가 제공하는 TTS(Text to Speech)를 이용하여 대답할 수 있습니다.

> **Notes**
>- 이 코드는 Python 3.11.4 커널에서 테스트 하였습니다.
>- 아래 코드는 PoC 용도입니다. 완벽한 코드가 아니므로 참고용으로 활용할 수 있습니다.
>- 해당 STT 및 TTS 기능을 활용하기 위해서는 마이크와 스피커와 같은 Hardware를 필요로 합니다. 개발환경이 컨테이너 기반일 경우, 정상 수행되지 않을 수 있습니다.

In [ ]:
# 데스크톱(로컬)에서 실행하는 경우 입니다. 아래와 같이 라이브러리 재설치가 필요할 수 있습니다.
# 먼저 파이썬 런타임을 설치해야 합니다. https://www.python.org/downloads/
%pip install azure-cognitiveservices-speech
%pip install azure-identity
%pip install -r m./requirements.txt

실행에 필요한 환경 변수를 가져옵니다. 아래에 필요한 정보를 미리 .env 파일에 저장합니다.

1. Azure OpenAI API 정보
2. Azure Speech API 정보
3. 의약품개요정보 API 정보 (https://developers.kakao.com/console/app)


In [3]:
import azure.cognitiveservices.speech as speechsdk
import os
import openai
import json
import requests
import pytz
from urllib import parse
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

# Azure OpenAI resource 정보를 설정합니다.
openai.api_type     = "azure"
openai.api_key      = "xxxxxxxxxxxxxxxxxx"
openai.api_base     = "https://<your-openai>.openai.azure.com/"
openai.api_version  = "2023-07-01-preview"
deployment_id       = "gpt-35-turbo-16k"
drug_api_key        = "xxxxxxxxxxxxxxxxxx"
speech_key          = "xxxxxxxxxxxxxxxxxx"
speech_region       = "xxxxx"
speech_language     = "ko-KR"

### Azure Speech To Text (STT) 엔진으로 사용자 명령을 텍스트로 수집

In [4]:
# Azure Cognitive Speech to Text 함수
def stt():
    # Creates a recognizer with the given settings
    # Azure STT & TTS API key
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region, speech_recognition_language='ko-KR')
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

    print("말씀 하세요~")

    # Starts speech recognition, and returns after a single utterance is recognized. The end of a
    # single utterance is determined by listening for silence at the end or until a maximum of 15
    # seconds of audio is processed.
    result = speech_recognizer.recognize_once()

    # Checks result.
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("음성인식결과: {}".format(result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("일치하는 음성이 없습니다.: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("음성 인식이 취소되었습니다.: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(
                cancellation_details.err+or_details))
    return result

### Azure Text To Speech (TTS) 엔진으로 텍스트를 읽음

In [ ]:
# Azure Cognitive Text to Speech 함수
def tts(input):
    print(input)
    # Set the voice name, refer to https://learn.microsoft.com/ko-kr/azure/ai-services/speech-service/language-support?tabs=tts for full list.
    # speech_config.speech_synthesis_voice_name = "ko-KR-InJoonNeural"
    # Creates a synthesizer with the given settings
    # Azure STT & TTS API key
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_synthesis_voice_name = "ko-KR-SeoHyeonNeural"
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

    # Synthesizes the received text to speech.
    result = speech_synthesizer.speak_text_async(input).get()

    # Checks result.
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized to speaker for text [{}]".format(input))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech synthesis canceled: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(
                    cancellation_details.error_details))
        print("Did you update the subscription info?")

### Function 1. 약 정보 찾기 API

In [15]:
# 약 정보 찾기 API
def get_drug_info(drug_name):
    headers = {
        "Content-Type": "application/json"
    }
    params = {
        "serviceKey": drug_api_key,
        "pageNo": 1,
        "numOfRows": 3,
        "itemName": drug_name,
        "type": "json"
    }
    url = "https://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["body"]["items"][0]
    return_data = {
        "efcyQesitm": response_summary["efcyQesitm"]
    }
    
    return json.dumps(return_data, ensure_ascii=False)


### Function 2. 약 사용법 찾기 API

In [16]:
# 약 사용법 찾기 API
def get_drug_usage(drug_name):
    headers = {
        "Content-Type": "application/json"
    }
    params = {
        "serviceKey": drug_api_key,
        "pageNo": 1,
        "numOfRows": 3,
        "itemName": drug_name,
        "type": "json"
    }
    url = "https://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["body"]["items"][0]
    return_data = {
        "useMethodQesitm": response_summary["useMethodQesitm"]
    }
    
    return json.dumps(return_data, ensure_ascii=False)


#### 약 정보 찾기 API 동작 체크

In [8]:
response = get_drug_info("타이레놀")
print(response)

{"itemName": "어린이타이레놀산160밀리그램(아세트아미노펜)", "efcyQesitm": "이 약은 감기로 인한 발열 및 동통(통증), 두통, 신경통, 근육통, 월경통, 염좌통(삔 통증), 치통, 관절통, 류마티양 동통(통증)에 사용합니다.\n"}


### OpenAI Function Calling에서 활용하려는 함수들을 정의

In [9]:
functions = [
        {
            "name": "get_drug_info",
            "description": "API that tells drug information by saying the drug name",
            "parameters": {
                "type": "object",
                "properties": {
                    "drug_name": {"type": "string"},        
                },
                "required": ["drug_name"],
            },
        },
        {
            "name": "get_drug_usage",
            "description": "API that tells how to use the drug by saying the drug name",
            "parameters": {
                "type": "object",
                "properties": {
                    "drug_name": {"type": "string"},        
                },
                "required": ["drug_name"]
            }
        }
    ]

available_functions = {
            "get_drug_info": get_drug_info,
            "get_drug_usage": get_drug_usage
        } 

#### 함수에 제공되는 매개변수가 맞는지 검수하는 함수

In [10]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

#### OpenAI GPT 모델을 이용하여 사용자 의도에 맞는 함수를 자동 선택하고 실행하는 함수

In [11]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT
    try:
        response = openai.ChatCompletion.create(
            deployment_id=deployment_id,
            messages=messages,
            functions=functions,
            function_call="auto", 
        )
    except Exception as e:
        print(e)


    # print(response)
    response_message = response["choices"][0]["message"]
    

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # print("Recommended Function call:")
        # print(response_message.get("function_call"))
        # print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        # print("Output of function call:")
        # print(function_response)
        # print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # function_name 값에 따른 분기 처리
        if function_name == "get_drug_info":
            messages.append(
                {"role": "system", "content": "You are a bot that provide drug information.",}
            )
        elif function_name == "get_drug_usage":
            messages.append(
                {"role": "system", "content": "You are a bot that tells you how to use the medicine.",}
            )
        else :
            messages.append(
                {"role": "system", "content": "You are an AI assistant that helps people find information. The answer must be judged and answered based on factual data. Please use simple expressions as much as possible."},
            )
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        # print("Messages in second request:")
        # for message in messages:
        # print(messages)
        # print(json.dumps(messages, ensure_ascii=False, indent=4))

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id,
            temperature=0
        )  # get a new response from GPT where it can see the function response
        
        # print("Second Call: ")
        # print(second_response)
        # print()

        return second_response

#### GPT 모델에 자연어 기반 질의하는 함수
해당 실습에서는 Function Calling에 해당되지 않는 요청하면 종료합니다.

In [12]:
def gpt(input):
    messages = [
        {"role": "user", "content": input}
    ]
    assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
    # assistant_response 값이 비어있을 경우
    if not assistant_response:
        return "제가 답변 드릴 수 있는 질문이 아닙니다. 다시 질문해주세요."
    else:
        content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
        content = content.replace("\\n", "\n").replace("\\\"", "\"")
        return content


GPT에 질의하여 각 기능이 정상적으로 동작하는지 살펴 봅니다.

In [17]:
drug_name = "타이레놀"
query = f"{drug_name}은 복용법 알려줘"

print(gpt(query))

"타이레놀은 다음과 같은 복용법을 따릅니다:

- 어린이타이레놀산 160밀리그램(아세트아미노펜)은 만 7~12세 소아를 대상으로 합니다.
- 1회 권장용량을 4~6시간마다 필요 시 물 없이 혀에 직접 복용합니다.
- 가능한 최단 기간 동안 최소 유효용량으로 복용하며, 1일 5회(75 mg/kg)를 초과하여 복용하지 않습니다.
- 몸무게를 아는 경우, 몸무게에 따른 용량(10~15 mg/kg)으로 복용하는 것이 더 적절합니다.
- 자세한 사항은 허가사항 상세정보를 참고하시기 바랍니다.

위의 정보는 어린이타이레놀산 160밀리그램(아세트아미노펜)에 대한 복용법입니다. 다른 제품의 경우, 제품에 표기된 사용법을 따르시기 바랍니다."


In [24]:
# if __name__ == "__main__":
tts("약의 정보, 복용법 등을 물어보세요.")

while True:
    result_stt = stt().text
    print(result_stt)
    if(result_stt == ""):
        # 음성 인식 실패
        print("음성 인식 실패")
        tts("음성인식에 실패했습니다. 다시 말씀해 주세요.")
    elif(result_stt == "나가기." or result_stt == "종료."):
        print("대화 종료")
        break
    else:
        # 음성인식 성공
        result_gpt = gpt(result_stt)
        print(result_gpt)
        tts(result_gpt)

Speech synthesized to speaker for text [약의 정보, 복용법 등을 물어보세요.]
말씀 하세요~
음성인식결과: 타이레놀은 어떤 약이야?
타이레놀은 어떤 약이야?
"타이레놀은 아세트아미노펜이라는 활성 성분을 포함한 약으로, 감기로 인한 발열, 동통(통증), 두통, 신경통, 근육통, 월경통, 염좌통(삔 통증), 치통, 관절통, 류마티양 동통(통증) 등을 완화하기 위해 사용됩니다."
Speech synthesized to speaker for text ["타이레놀은 아세트아미노펜이라는 활성 성분을 포함한 약으로, 감기로 인한 발열, 동통(통증), 두통, 신경통, 근육통, 월경통, 염좌통(삔 통증), 치통, 관절통, 류마티양 동통(통증) 등을 완화하기 위해 사용됩니다."]
말씀 하세요~
음성인식결과: 종료.
종료.
대화 종료


이제부터는 **여러분의 API** 를 하나씩 추가해 보세요.
수고하셨습니다.